# DEL 3.5: Oppgaver med _restcountries_ (del 1)

Her kommer noen oppgaver der du skal bruke `pydantic`-modeller for å strukturere
data og hente ut informasjon fra _restcountries_-apiet. 


In [7]:
# Importer bibliotekene vi trenger
import re
import requests
from pydantic import BaseModel
from urllib.parse import quote


# For automatiske tester trenger vi pytest og ipytest
import ipytest
import pytest

ipytest.autoconfig()

In [8]:
class CountryNameEntry(BaseModel):
    common: str
    official: str


class CountryName(CountryNameEntry):
    nativeName: dict[str, CountryNameEntry]


class Currency(BaseModel):
    name: str
    symbol: str


class Country(BaseModel):
    name: CountryName
    population: int
    area: float
    region: str
    subregion: str
    languages: dict[str, str]
    currencies: dict[str, Currency]

Det kan være du må redefinere `pydantic`-modellene i oppgavene, dersom du har behov 
for å jobbe med data som ikke er inkludert i definisjoner fra tidligere celler. 



Under er en funksjon du kan bruke for å hente landdata fra api-et. Kjør cellen under
for å laste funksjonen, og bruk den i senere oppgaver.

In [9]:

def hent_landdata(landnavn: str) -> Country:
    """Henter landdata fra restcountries API og returnerer et Country objekt."""
    url = f"https://restcountries.com/v3.1/name/{quote(landnavn)}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        country_data = data[0]
        country = Country(**country_data)
        return country
    else:
        raise Exception(f"Error fetching country data: {response.status_code}")
    
# Eksempel på bruk:
norway = hent_landdata("norway")
print(norway.model_dump_json(indent=2))

{
  "name": {
    "common": "Norway",
    "official": "Kingdom of Norway",
    "nativeName": {
      "nno": {
        "common": "Noreg",
        "official": "Kongeriket Noreg"
      },
      "nob": {
        "common": "Norge",
        "official": "Kongeriket Norge"
      },
      "smi": {
        "common": "Norgga",
        "official": "Norgga gonagasriika"
      }
    }
  },
  "population": 5606944,
  "area": 386224.0,
  "region": "Europe",
  "subregion": "Northern Europe",
  "languages": {
    "nno": "Norwegian Nynorsk",
    "nob": "Norwegian Bokmål",
    "smi": "Sami"
  },
  "currencies": {
    "NOK": {
      "name": "Norwegian krone",
      "symbol": "kr"
    }
  }
}


## Oppgave 3.5-1

- Lag en funksjon som tar inn et landnavn og returnerer populasjonen i landet. 
- Lag en funksjon som tar inn en liste med navn og returnerer navnet på landet med høyest populasjon.
- Lag en funksjon som tar inn en liste med navn og returnerer en liste av tuples
  der den første verdien er landets navn og den andre verdien er landets
  populasjon. Listen skal være sortert etter populasjon, i synkende rekkefølge.
- Skriv ut en liste (med `print`) over navn og populasjon til landene Sverige,
Frankrike, Tyskland og Storbritannia, i synkende rekkefølge 

Bruk startkoden under, og hent data fra api-et med funksjonen `hent_landdata`

In [23]:
# Ved behov: Redefiner pydantic Country eller andre modeller her:
# class Country(BaseModel):
#     ...

def hent_populasjon(landnavn: str) -> int:
    """Henter befolkningstall for et land fra restcountries API."""
    populasjon = hent_landdata(landnavn).population
    return populasjon


def hoyest_befolkning(landnavn_liste: list[str]) -> str:
    """Returnerer navnet på landet med høyest befolkning fra en liste av landnavn."""
    høyest_befolkning = landnavn_liste[0]
    for landnavn in landnavn_liste:
        populasjon = hent_landdata(landnavn).population
        if populasjon > hent_landdata(høyest_befolkning).population:
            høyest_befolkning = landnavn
    return høyest_befolkning


def hent_populasjon_sortert(landnavn_liste: list[str]) -> list[tuple[str, int]]:
    """Returnerer en liste av tuples med landnavn og befolkning, sortert etter befolkning."""
    populasjon_liste = []
    for landnavn in landnavn_liste:
        populasjon = hent_landdata(landnavn).population
        populasjon_liste.append((landnavn, populasjon))
    populasjon_liste.sort(key=lambda x: x[1], reverse=True)
    return populasjon_liste
    
    
def oppgave_befolkning():
    """Skriver ut liste over land og befolkning, sortert etter befolkning."""
    landnavn = ("germany", "france", "sweden", "united kingdom")  # Fyll inn flere landnavn fra oppgaven her
    hent_populasjon_sortert(landnavn)
    resultat = hent_populasjon_sortert(landnavn)
    return resultat


# Denne linjen gjør at oppgave_befolkning funksjonen kjøres når du kjører cellen
oppgave_befolkning()

[('germany', 83491249),
 ('united kingdom', 69281437),
 ('france', 66351959),
 ('sweden', 10605098)]

Kjør cellen under for å sjekke svaret ditt:

In [24]:
%%ipytest --tb=short
import re

def get_country_data(name):
    response = requests.get(f"https://restcountries.com/v3.1/name/{quote(name)}")
    data = response.json()
    country_data = data[0]
    country = Country(**country_data)
    return country


def test_model():
    assert set(Country.model_fields.keys()).issuperset({"population", "name"}), (
        "Country modellen mangler forventede egenskaper (population / name)"
    )


def test_hent_populasjon():
    norway = get_country_data("norway")
    estonia = get_country_data("estonia")
    assert isinstance(hent_populasjon("norge"), int), (
        "Får ikke forventet datatype fra hent_populasjon"
    )
    assert hent_populasjon("norway") == norway.population, (
        "Får ikke forventet befolkningstall fra hent_populasjon"
    )
    assert hent_populasjon("estonia") == estonia.population, (
        "Får ikke forventet befolkningstall fra hent_populasjon"
    )


def test_hoyest_befolkning():
    landnavn = ["united kingdom", "japan"]
    assert hoyest_befolkning(landnavn).lower() == "japan", (
        "Får ikke forventet resultat fra hoyest_befolkning"
    )


def test_befolkning_sortert():
    landnavn = ["sweden", "france", "germany"]
    resultat = hent_populasjon_sortert(landnavn)
    assert isinstance(resultat, list), (
        "Får ikke forventet datatype fra hent_populasjon_sortert"
    )
    assert all(isinstance(tup, tuple) and len(tup) == 2 for tup in resultat), (
        "Får ikke forventet liste av tuples fra hent_populasjon_sortert"
    )
    forventet_rekkefolge = ["germany", "france", "sweden"]
    faktisk_rekkefolge = [land.lower().strip() for land, _ in resultat]
    assert faktisk_rekkefolge == forventet_rekkefolge, (
        "Får ikke forventet sortering i hent_populasjon_sortert"
    )


def test_oppgave_befolkning(capsys):
    oppgave_befolkning()
    captured = capsys.readouterr()
    country_population_match_1 = re.findall(
        r"(germany|france|sweden|united kingdom).+?(\d+)", captured.out, re.I | re.S
    )
    country_population_match_2 = re.findall(
        r"(\d+).+?(germany|france|sweden|united kingdom)", captured.out, re.I | re.S
    )
    country_population_match = (
        country_population_match_1
        if len(country_population_match_1) > len(country_population_match_2)
        else country_population_match_2
    )
    assert len(country_population_match) == 4, (
        "Finner ikke forventet antall land med befolkningstall i oppgave_befolkning utskrift"
    )
    data = {
        country.name.common: country.population
        for name in ["germany", "france", "sweden", "united kingdom"]
        for country in [get_country_data(name)]
    }
    data_sorted = list(data.items())
    data_sorted.sort(key=lambda x: x[1], reverse=True)
    got_data = [(name, int(pop)) for name, pop in country_population_match]
    got_keys = [name for name, _ in got_data]
    assert set(data.keys()) == set(got_keys), (
        "Finner ikke forventede land i oppgave_befolkning utskrift"
    )
    got_pops = [pop for _, pop in got_data]
    assert set(got_pops) == set(data.values()), (
        "Finner ikke forventede befolkningstall i oppgave_befolkning utskrift"
    )

    assert got_data == data_sorted, (
        "Får ikke forventet sortering i oppgave_befolkning utskrift"
    )

....F                                                                                        [100%]
============================================ FAILURES =============================================
_____________________________________ test_oppgave_befolkning _____________________________________
C:\Users\hasse\AppData\Local\Temp\ipykernel_25004\1341584509.py:68: in test_oppgave_befolkning
    assert len(country_population_match) == 4, (
E   AssertionError: Finner ikke forventet antall land med befolkningstall i oppgave_befolkning utskrift
E   assert 0 == 4
E    +  where 0 = len([])
===================================== short test summary info =====================================
FAILED t_717cdc2b024f44bebd31d4f5f63642b0.py::test_oppgave_befolkning - AssertionError: Finner ikke forventet antall land med befolkningstall i oppgave_befolkning utsk...
1 failed, 4 passed in 10.54s


## Oppgave 3.5-2

Lag en `pydantic` modell med navn `Postnummer` som har egenskapene `nummer`
(str) og `land` (str). Land skal ha standardverdi `"Norge"`. 

In [ ]:
class _(_): # Definer modellen her (bruk eksakt navn fra oppgaven. Husk riktig arv!)
    pass  # Fyll inn implementasjon her

Kjør cellen under for å sjekke svaret ditt:

In [ ]:
%%ipytest --tb=short 


def test_postnummer_model():
    assert Postnummer is not None, (
        "Postnummer modellen er ikke definert - har du brukt riktig navn? "
        + "Husk også å kjøre cellen med din kode før du kjører testen."
    )
    assert issubclass(Postnummer, BaseModel), (
        "Feil i modellen - Postnummer må arve fra pydantic BaseModel."
    )
    assert Postnummer.model_fields.keys() == {"nummer", "land"}, (
        "Feil i modellen - Postnummer har ikke de forventede egenskapene "
        + "(nummer og land), eller har feil navn på egenskapene."
    )
    assert Postnummer.model_fields["nummer"].annotation == str, (
        "Feil i modellen - egenskapen nummer må være av typen str."
    )
    assert Postnummer.model_fields["land"].annotation == str, (
        "Feil i modellen - egenskapen land må være av typen str."
    )
    assert Postnummer.model_fields["land"].default == "Norge", (
        "Feil i modellen - standardverdien for egenskapen land må være 'Norge'."
    )
    postnr = Postnummer(nummer="5000")
    assert postnr.nummer == "5000", (
        "Feil i modellen - egenskapen nummer fungerer ikke som forventet."
    )
    assert postnr.land == "Norge", (
        "Feil i modellen - standardverdien for egenskapen land fungerer ikke som forventet."
    )
    postnr2 = Postnummer(nummer="7500", land="Sverige")
    assert postnr2.land == "Sverige", (
        "Feil i modellen - egenskapen land fungerer ikke som forventet."
    )

## Oppgave 3.5-3

Lag en funksjon, valider_postnummer, som tar inn et objekt `Postnummer` (som du
definerte i forrige oppgave), og sjekker om objektets verdi for nummer er gyldig,
basert på informasjon fra _restcountries_ api-et.

> Tips: 
>
> Sjekk ut egenskapen `postalCode` i `Country` modellen og `PostalCode` modellen 
> definert tidligere.

**Regex**

Her kommer en liten oppfriskning til regex i python. Man bruker regex med det 
innebygde (trenger ikke installeres med pip) biblioteket `re` som importeres 
med `import re`. Dette biblioteket skal allerede være importert om du har 
kjørt første kodecelle i denne notebooken. 

- Finn første: `re.search(pattern, text)`
- Finn alle: `re.findall(pattern, text)`
- Finn og erstatt: `re.sub(pattern, replace_with, text)`

```python
# Eksempel med grunnleggende bruk av re-biblioteket

import re # Importer biblioteket

# Regex-mønster og tekst det skal søkes i
mail_ptrn = r'\S+@\S+\.\S+' # Bruk raw string (r'...') for å unngå escape-char feil
mail_ptrn_alt = '\\S+@\\S+\\.\\S+' # Du kan også bruke vanlig string med escape-tegn
                                   # (her: \\ i stedet for \)
text = "Kontakt oss på post@eksempel.no eller support@firma.com"

# Finn alle e-postadresser
mails = re.findall(mail_ptrn, text)
print(mails)  # ['post@eksempel.no', 'support@firma.com']

# Finn første treff
mail = re.search(mail_ptrn, text)
print(mail.group()) # post@eksempel.no

# Dette kan også brukes til å sjekke om det finnes
# en match for mønster i tekst. 
# (None returneres fra re.search om ingen treff)
if re.search(mail_ptrn, text):
    print("Fant e-postadresse!")

# Erstatt
# (Kan definere mønster direkte i funksjonshode om ønskelig)
ny_text = re.sub(r'\S+@\S+\.\S+', '[SKJULT]', text)
print(ny_text)  # "Kontakt oss på [SKJULT] eller [SKJULT]"
```

In [ ]:
def valider_postnummer(postnummer: Postnummer) -> bool:
    """Validerer at postnummeret følger forventet format.
    
    Skal fungere for landene norge, united kingdom, og germany.
    """
    pass # Fyll inn implementasjon her

# Her er noen eksempler på postnummer som kan testes, og skal være gyldige
test_postnummer = {
    "United Kingdom": "EH11AA", # Anta uk har 2 bokstaver, 2 tall, så 2 bokstaver
    "Norway": "5000",
    "Germany": "50667",
}    

Kjør cellen under for å sjekke svaret ditt:

In [ ]:
%%ipytest --tb=short

def test_valider_postnummer():
    valid_codes = {
        "United Kingdom": "EH11AA",
        "Norge": "5000",
        "Germany": "50667",
    }
    invalid_codes = {
        "United Kingdom": "12345",
        "Norge": "ABCDE",
        "Germany": "XYZ987",
    }
    for land, kode in valid_codes.items():
        postnr = Postnummer(nummer=kode, land=land)
        assert valider_postnummer(postnr), (
            f"Postnummer {kode} i {land} skal være gyldig, men ble vurdert som ugyldig."
        )
    for land, kode in invalid_codes.items():
        postnr = Postnummer(nummer=kode, land=land)
        assert not valider_postnummer(postnr), (
            f"Postnummer {kode} i {land} skal være ugyldig, men ble vurdert som gyldig."
        )